## This is the softmax workbook for ECE C147/C247 Assignment #2

Please follow the notebook linearly to implement a softmax classifier.

Please print out the workbook entirely when completed.

The goal of this workbook is to give you experience with training a softmax classifier.

In [1]:
import random
import numpy as np
from utils.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [107]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = '/home/andrea/git/UCLA/UCLA_ECE147/Homework2/HW2_code/cifar-10-batches-py' # You need to update this line
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Training a softmax classifier.

The following cells will take you through building a softmax classifier.  You will implement its loss function, then subsequently train it with gradient descent.  Finally, you will choose the learning rate of gradient descent to optimize its classification performance.

In [108]:
from nndl import Softmax

In [109]:
# Declare an instance of the Softmax class.  
# Weights are initialized to a random value.
# Note, to keep people's first solutions consistent, we are going to use a random seed.

np.random.seed(1)

num_classes = len(np.unique(y_train))
num_features = X_train.shape[1]

softmax = Softmax(dims=[num_classes, num_features])

#### Softmax loss

In [129]:
## Implement the loss function of the softmax using a for loop over
#  the number of examples

unit_loss = softmax.loss(X_train, y_train)

In [120]:
print(loss)

2.3277607028048966


In [115]:
np.exp(-2.3)

0.10025884372280375

## Question: 

You'll notice the loss returned by the softmax is about 2.3 (if implemented correctly).  Why does this make sense?

## Answer:

The computed loss is the average loss per observation. The loss is $-2.3$, so the softmax per observation is $e^{-2.3}$, or about $\frac{1}{10}$. This makes perfect sense since $W$ is approximately $0$, so the softmax evaluates to $\textrm{softmax}_j(x^{i}) = \frac{e^{w_j^T x}}{\sum_{k=1}^{c} e^{w_k^T x}} \approx \frac{e^{0}}{\sum_{k=1}^{c} e^{0}} = \frac{1}{c} = \frac{1}{10}$ since we have $c=10$ classes.

#### Softmax gradient

In [261]:
## Calculate the gradient of the softmax loss in the Softmax class.
# For convenience, we'll write one function that computes the loss
#   and gradient together, softmax.loss_and_grad(X, y)
# You may copy and paste your loss code from softmax.loss() here, and then
#   use the appropriate intermediate values to calculate the gradient.

loss, grad = softmax.loss_and_grad(X_dev,y_dev)

# Compare your gradient to a gradient check we wrote. 
# You should see relative gradient errors on the order of 1e-07 or less if you implemented the gradient correctly.
softmax.grad_check_sparse(X_dev, y_dev, grad)

numerical: -0.412513 analytic: -0.412513, relative error: 3.100247e-08
numerical: 1.449105 analytic: 1.449105, relative error: 5.650975e-09
numerical: 2.770640 analytic: 2.770640, relative error: 2.425229e-09
numerical: -1.671566 analytic: -1.671566, relative error: 6.438381e-09
numerical: 2.540387 analytic: 2.540387, relative error: 9.276612e-09
numerical: 1.122858 analytic: 1.122858, relative error: 3.049509e-08
numerical: -0.832407 analytic: -0.832407, relative error: 2.164703e-08
numerical: 0.362969 analytic: 0.362969, relative error: 1.164066e-07
numerical: 0.110281 analytic: 0.110281, relative error: 2.015857e-09
numerical: 1.501076 analytic: 1.501076, relative error: 4.027766e-09


In [254]:
W = softmax.W
y = y_train
X = X_train

# def softmax(c, x):
#     return np.exp(W[c].T@x)/(np.exp(x@W.T).sum(axis=1))

# loss -= np.log(softmax(y, X))

# loss /= X.shape[0]
# softmax(y, X)
loss = (-np.log((np.exp((W[y]*X).sum(axis=1))/(np.exp(X@W.T).sum(axis=1))))).mean()
softmax_mat = ((np.exp(X@W.T))/(np.exp(X@W.T).sum(axis=1)[:,np.newaxis]))
indicator_func = 1*(y[:,np.newaxis] == np.arange(10))
fast_grad = (1.0/X.shape[0])*(softmax_mat - indicator_func).T@X
fast_grad

array([[-6.74229960e-02, -1.11687303e+00, -3.05855272e+00, ...,
        -1.78854575e-01, -1.45885558e+00,  1.08666450e-04],
       [-1.06544942e+00, -7.02928086e-01, -8.98941182e-01, ...,
        -1.26895409e+00, -2.02551435e+00,  3.44880455e-04],
       [ 4.27472812e-02,  5.59734898e-02,  1.25114756e+00, ...,
         1.19258009e-01,  9.15453440e-01, -1.27094756e-03],
       ...,
       [-7.53409582e-01, -8.82949360e-01, -1.03312344e+00, ...,
        -6.49982586e-01,  6.54976809e-01, -1.88552556e-04],
       [-1.59487390e+00, -2.73922820e+00, -4.70980699e+00, ...,
         7.74675600e-01, -1.24149185e+00, -1.61897274e-03],
       [-4.84774260e+00, -5.45388202e+00, -6.76998403e+00, ...,
        -1.74281117e+00, -2.32992368e+00,  1.88039082e-03]])

In [216]:
((np.exp(X[0]@W.T))/(np.exp(X[0]@W.T)[np.newaxis,:].sum(axis=1)[:,np.newaxis]))

array([[0.0901499 , 0.11439849, 0.10781625, 0.07082191, 0.10381199,
        0.09467076, 0.07340523, 0.10914627, 0.12930551, 0.1064737 ]])

In [217]:
(-1*(y[:,np.newaxis] == np.arange(10)))[0]

array([ 0,  0,  0,  0,  0,  0, -1,  0,  0,  0])

In [237]:
row_smax=((np.exp(X[0]@W.T))/(np.exp(X[0]@W.T)[np.newaxis,:].sum(axis=1)[:,np.newaxis]))
row_indic=(1*(y[:,np.newaxis] == np.arange(10)))[0]
row_grad= (1.0/X.shape[0])*(row_smax - row_indic).T@X[0][np.newaxis,:]
row_grad

array([[-1.31806320e-04, -1.36111137e-04, -1.27817685e-04, ...,
        -6.22990228e-05, -7.80064689e-05,  1.83979381e-06],
       [-1.67259692e-04, -1.72722423e-04, -1.62198191e-04, ...,
        -7.90562651e-05, -9.89887130e-05,  2.33466306e-06],
       [-1.57635935e-04, -1.62784351e-04, -1.52865661e-04, ...,
        -7.45075401e-05, -9.32931184e-05,  2.20033164e-06],
       ...,
       [-1.59580536e-04, -1.64792464e-04, -1.54751416e-04, ...,
        -7.54266675e-05, -9.44439853e-05,  2.22747500e-06],
       [-1.89054941e-04, -1.95229508e-04, -1.83333886e-04, ...,
        -8.93579161e-05, -1.11887718e-04,  2.63888795e-06],
       [-1.55673015e-04, -1.60757322e-04, -1.50962141e-04, ...,
        -7.35797547e-05, -9.21314106e-05,  2.17293259e-06]])

In [256]:
# def softmax(c, x):
#     return np.exp(self.W[c]@x)/(np.exp(self.W@x)).sum()

# for i in range(0, X.shape[0]):
#     for k in range(0, self.W.shape[0]):
#         dL_i_dw_k = (softmax(k, X[i]) - (k == y[i]))*X[i]
#         grad[k,:] += dL_i_dw_k
        
# grad /= X.shape[0]

test_grad2 = np.zeros_like(W)

def softmax_func(c, x):
    return np.exp(W[c]@x)/(np.exp(W@x)).sum()

for i in range(0, X.shape[0]):
    for k in range(0, W.shape[0]):
        dL_i_dw_k = (softmax_func(k, X[i]) - (k == y[i]))*X[i]
        test_grad2[k,:] += dL_i_dw_k
        
test_grad2 /= X.shape[0]
test_grad2

array([[-6.74229960e-02, -1.11687303e+00, -3.05855272e+00, ...,
        -1.78854575e-01, -1.45885558e+00,  1.08666450e-04],
       [-1.06544942e+00, -7.02928086e-01, -8.98941182e-01, ...,
        -1.26895409e+00, -2.02551435e+00,  3.44880455e-04],
       [ 4.27472812e-02,  5.59734898e-02,  1.25114756e+00, ...,
         1.19258009e-01,  9.15453440e-01, -1.27094756e-03],
       ...,
       [-7.53409582e-01, -8.82949360e-01, -1.03312344e+00, ...,
        -6.49982586e-01,  6.54976809e-01, -1.88552556e-04],
       [-1.59487390e+00, -2.73922820e+00, -4.70980699e+00, ...,
         7.74675600e-01, -1.24149185e+00, -1.61897274e-03],
       [-4.84774260e+00, -5.45388202e+00, -6.76998403e+00, ...,
        -1.74281117e+00, -2.32992368e+00,  1.88039082e-03]])

In [250]:
def softmax_func(c, x):
    return np.exp(W[c]@x)/(np.exp(W@x)).sum()

test_grad = np.zeros_like(W)

for i in range(0, X.shape[0]):
    for k in range(0, W.shape[0]):
#         print(f"{i=}, {softmax_func(k, X[i])=}")
#         print(f"{i=}, {-1*(k == y[i])=}")
        dL_i_dw_k = (softmax_func(k, X[i]) - (k == y[i]))*X[i]
#         assert grad[k,:].shape == dL_i_dw_k.shape, f"{grad[k,:].shape =} != {dL_i_dw_k.shape =}"
        test_grad[k,:] += dL_i_dw_k

test_grad /= X.shape[0]
test_grad

array([[-6.74229960e-02, -1.11687303e+00, -3.05855272e+00, ...,
        -1.78854575e-01, -1.45885558e+00,  1.08666450e-04],
       [-1.06544942e+00, -7.02928086e-01, -8.98941182e-01, ...,
        -1.26895409e+00, -2.02551435e+00,  3.44880455e-04],
       [ 4.27472812e-02,  5.59734898e-02,  1.25114756e+00, ...,
         1.19258009e-01,  9.15453440e-01, -1.27094756e-03],
       ...,
       [-7.53409582e-01, -8.82949360e-01, -1.03312344e+00, ...,
        -6.49982586e-01,  6.54976809e-01, -1.88552556e-04],
       [-1.59487390e+00, -2.73922820e+00, -4.70980699e+00, ...,
         7.74675600e-01, -1.24149185e+00, -1.61897274e-03],
       [-4.84774260e+00, -5.45388202e+00, -6.76998403e+00, ...,
        -1.74281117e+00, -2.32992368e+00,  1.88039082e-03]])

In [229]:
grad

array([[-1.31806320e-04, -1.36111137e-04, -1.27817685e-04, ...,
        -6.22990228e-05, -7.80064689e-05,  1.83979381e-06],
       [-1.67259692e-04, -1.72722423e-04, -1.62198191e-04, ...,
        -7.90562651e-05, -9.89887130e-05,  2.33466306e-06],
       [-1.57635935e-04, -1.62784351e-04, -1.52865661e-04, ...,
        -7.45075401e-05, -9.32931184e-05,  2.20033164e-06],
       ...,
       [-1.59580536e-04, -1.64792464e-04, -1.54751416e-04, ...,
        -7.54266675e-05, -9.44439853e-05,  2.22747500e-06],
       [-1.89054941e-04, -1.95229508e-04, -1.83333886e-04, ...,
        -8.93579161e-05, -1.11887718e-04,  2.63888795e-06],
       [-1.55673015e-04, -1.60757322e-04, -1.50962141e-04, ...,
        -7.35797547e-05, -9.21314106e-05,  2.17293259e-06]])

In [247]:
row_smax=((np.exp(X@W.T))/(np.exp(X@W.T).sum(axis=1)[:,np.newaxis]))
row_indic=(1*(y[:,np.newaxis] == np.arange(10)))
row_grad = (1.0/X.shape[0])*(row_smax - row_indic).T@X
row_grad

array([[-6.74229960e-02, -1.11687303e+00, -3.05855272e+00, ...,
        -1.78854575e-01, -1.45885558e+00,  1.08666450e-04],
       [-1.06544942e+00, -7.02928086e-01, -8.98941182e-01, ...,
        -1.26895409e+00, -2.02551435e+00,  3.44880455e-04],
       [ 4.27472812e-02,  5.59734898e-02,  1.25114756e+00, ...,
         1.19258009e-01,  9.15453440e-01, -1.27094756e-03],
       ...,
       [-7.53409582e-01, -8.82949360e-01, -1.03312344e+00, ...,
        -6.49982586e-01,  6.54976809e-01, -1.88552556e-04],
       [-1.59487390e+00, -2.73922820e+00, -4.70980699e+00, ...,
         7.74675600e-01, -1.24149185e+00, -1.61897274e-03],
       [-4.84774260e+00, -5.45388202e+00, -6.76998403e+00, ...,
        -1.74281117e+00, -2.32992368e+00,  1.88039082e-03]])

In [252]:
grad

array([[ 4.02876447e-01, -9.66425599e-02, -1.47020505e+00, ...,
         5.56593389e-01, -5.64809866e-01,  3.34246868e-02],
       [-4.11454543e-02, -1.68827085e-01, -3.43605743e-01, ...,
        -7.23651591e-01, -1.29233786e+00,  1.16444900e-02],
       [ 4.98903566e-01, -8.15755453e-02,  1.55088135e+00, ...,
         1.74023186e-01,  1.34731015e+00,  1.09759142e-02],
       ...,
       [-2.60863521e+00, -2.10003201e+00, -2.14235096e+00, ...,
        -8.49511818e-01,  1.76368126e-01,  2.01447899e-03],
       [ 1.31455161e-01, -1.67418405e+00, -4.34702705e+00, ...,
         1.98801118e+00, -6.44388349e-01, -9.17433601e-03],
       [-6.02646288e+00, -6.55550617e+00, -7.64222320e+00, ...,
        -1.56708886e+00, -2.02956084e+00,  7.77870268e-03]])

In [253]:
test_grad

array([[-6.74229960e-02, -1.11687303e+00, -3.05855272e+00, ...,
        -1.78854575e-01, -1.45885558e+00,  1.08666450e-04],
       [-1.06544942e+00, -7.02928086e-01, -8.98941182e-01, ...,
        -1.26895409e+00, -2.02551435e+00,  3.44880455e-04],
       [ 4.27472812e-02,  5.59734898e-02,  1.25114756e+00, ...,
         1.19258009e-01,  9.15453440e-01, -1.27094756e-03],
       ...,
       [-7.53409582e-01, -8.82949360e-01, -1.03312344e+00, ...,
        -6.49982586e-01,  6.54976809e-01, -1.88552556e-04],
       [-1.59487390e+00, -2.73922820e+00, -4.70980699e+00, ...,
         7.74675600e-01, -1.24149185e+00, -1.61897274e-03],
       [-4.84774260e+00, -5.45388202e+00, -6.76998403e+00, ...,
        -1.74281117e+00, -2.32992368e+00,  1.88039082e-03]])

## A vectorized version of Softmax

To speed things up, we will vectorize the loss and gradient calculations.  This will be helpful for stochastic gradient descent.

In [ ]:
import time

In [ ]:
## Implement softmax.fast_loss_and_grad which calculates the loss and gradient
#    WITHOUT using any for loops.  

# Standard loss and gradient
tic = time.time()
loss, grad = softmax.loss_and_grad(X_dev, y_dev)
toc = time.time()
print('Normal loss / grad_norm: {} / {} computed in {}s'.format(loss, np.linalg.norm(grad, 'fro'), toc - tic))

tic = time.time()
loss_vectorized, grad_vectorized = softmax.fast_loss_and_grad(X_dev, y_dev)
toc = time.time()
print('Vectorized loss / grad: {} / {} computed in {}s'.format(loss_vectorized, np.linalg.norm(grad_vectorized, 'fro'), toc - tic))

# The losses should match but your vectorized implementation should be much faster.
print('difference in loss / grad: {} /{} '.format(loss - loss_vectorized, np.linalg.norm(grad - grad_vectorized)))

# You should notice a speedup with the same output.

## Stochastic gradient descent

We now implement stochastic gradient descent.  This uses the same principles of gradient descent we discussed in class, however, it calculates the gradient by only using examples from a subset of the training set (so each gradient calculation is faster).

In [ ]:
# Implement softmax.train() by filling in the code to extract a batch of data
# and perform the gradient step.
import time


tic = time.time()
loss_hist = softmax.train(X_train, y_train, learning_rate=1e-7,
                      num_iters=1500, verbose=True)
toc = time.time()
print('That took {}s'.format(toc - tic))

plt.plot(loss_hist)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.show()

### Evaluate the performance of the trained softmax classifier on the validation data.

In [ ]:
## Implement softmax.predict() and use it to compute the training and testing error.

y_train_pred = softmax.predict(X_train)
print('training accuracy: {}'.format(np.mean(np.equal(y_train,y_train_pred), )))
y_val_pred = softmax.predict(X_val)
print('validation accuracy: {}'.format(np.mean(np.equal(y_val, y_val_pred)), ))

## Optimize the softmax classifier

In [ ]:
np.finfo(float).eps

In [ ]:
# ================================================================ #
# YOUR CODE HERE:
#   Train the Softmax classifier with different learning rates and 
#     evaluate on the validation data.
#   Report:
#     - The best learning rate of the ones you tested.  
#     - The best validation accuracy corresponding to the best validation error.
#
#   Select the SVM that achieved the best validation error and report
#     its error rate on the test set.
# ================================================================ #

# ================================================================ #
# END YOUR CODE HERE
# ================================================================ #
